In [1]:
import pandas as pd
import datetime
import traceback
import typing
import os
import json
from numpy import nan
from IPython.display import display

In [39]:
# PARAMETERS
ts_mili = "MILISECOND"
ts_mikro = "MIKROSECOND"

breakdown_col = {
    "c": "data",
    "u": "set",
    "d": "set"
}

# CUSTOM FUNCTIONS
def convert_timestamp(ts:int,unit:str=ts_mili) -> datetime.datetime:
    if unit == ts_mikro:
        ts /= 1000
    try:
        return datetime.datetime.fromtimestamp(ts)
    except Exception as e:
        traceback.print_exc()


def get_list_of_json(dir:str) -> typing.List[dict]:
    return [json.load(open(os.path.join(dir,x))) for x in os.listdir(dir)]


def separate_update_create(data:typing.List[dict]) -> typing.Dict[str,typing.List[dict]]:
    separate_dict = {"c": [], "u":[], "d":[]}
    for x in data:
        separate_dict[x["op"]].append(x)
    return separate_dict    


def create_df(data:typing.Dict[str,typing.List[dict]], data_col:dict=breakdown_col, data_col_name:str="data") -> pd.DataFrame:
    df_collection = {}
    for key, val in data.items():
        if len(val) == 0:
            df_collection[key] = pd.DataFrame()
            continue
        raw_df = pd.DataFrame(val)
        break_df = pd.DataFrame(raw_df[data_col[key]].to_list())
        combine_df = pd.concat([raw_df,break_df], axis=1).reset_index(drop=True)
        combine_df.rename(columns={data_col[key]:data_col_name},inplace=True)
        df_collection[key] = combine_df
    final_df = pd.concat(list(df_collection.values()), axis=0).reset_index(drop=True)
    return final_df


def load_data(dir:str, timestamp_col:typing.Dict[str,str]={}) -> pd.DataFrame:
    print("Start working ", dir)
    list_of_json = get_list_of_json(dir)
    data_list = separate_update_create(list_of_json)
    final_data = create_df(data_list)
    if timestamp_col != {}:
        for col, unit in timestamp_col.items():
            final_data[col] = final_data[col].apply(lambda x:convert_timestamp(x,unit))
    # final_data.fillna(method="ffill", inplace=True)
    print("Done working ", dir)
    return final_data


def join_table(left_df:pd.DataFrame, right_df:pd.DataFrame, 
               left_on:str,right_on:str, how:str="inner",
               suffix:tuple=('_left', '_right')) -> pd.DataFrame:
    return pd.merge(left_df,right_df,left_on=left_on,right_on=right_on,how=how,suffixes=suffix)


def add_value(x:int,y:int)->int:
    if pd.isna(x) and pd.isna(y):
        return nan
    else:
        return (x if not pd.isna(x) else 0) + (y if not pd.isna(y) else 0)


def main():
    data_collection = {
        "cards": {
            "url": "data/cards/",
            "data": None
        },
        "accounts": {
            "url": "data/accounts/",
            "data": None
        },
        "savings": {
            "url": "data/savings_accounts/",
            "data": None
        }
    }
    for key, val in data_collection.items():
        data_collection[key]["data"] = load_data(val["url"],{"ts": ts_mikro})
    
    # Card self join
    create_df = data_collection["cards"]["data"][data_collection["cards"]["data"]["op"] == "c"]
    update_df = data_collection["cards"]["data"][data_collection["cards"]["data"]["op"] == "u"]
    self_update_df = join_table(update_df,create_df[["id","card_id","card_number"]],"id","id",how="left")
    self_update_df = self_update_df.loc[:, ~self_update_df.columns.isin(["card_id_left","card_number_left"])]
    self_update_df.rename(columns={"card_id_right":"card_id","card_number_right":"card_number"},inplace=True)
    # data_collection["cards"]["data"] = pd.concat([create_df,self_update_df],axis=0).reset_index(drop=True)
    data_collection["cards"]["data"] = pd.concat([create_df,self_update_df],axis=0).sort_values(
                                          by=["ts"],ascending=[True]).reset_index(
                                          drop=True)
    # data_collection["cards"]["data"].fillna(method="ffill", inplace=True)

    # Account self join
    create_df = data_collection["accounts"]["data"][data_collection["accounts"]["data"]["op"] == "c"]
    update_df = data_collection["accounts"]["data"][data_collection["accounts"]["data"]["op"] == "u"]
    self_update_df = join_table(update_df,create_df[["id","account_id"]],"id","id",how="left")
    self_update_df = self_update_df.loc[:, ~self_update_df.columns.isin(["account_id_left"])]
    self_update_df.rename(columns={"account_id_right":"account_id"},inplace=True)
    data_collection["accounts"]["data"] = pd.concat([create_df,self_update_df],axis=0).sort_values(
                                          by=["ts"],ascending=[True]).reset_index(
                                          drop=True)
    # data_collection["accounts"]["data"].fillna(method="ffill", inplace=True)

    
    # Saving self join
    create_df = data_collection["savings"]["data"][data_collection["savings"]["data"]["op"] == "c"]
    update_df = data_collection["savings"]["data"][data_collection["savings"]["data"]["op"] == "u"]
    self_update_df = join_table(update_df,create_df[["id","savings_account_id"]],"id","id",how="left")
    self_update_df = self_update_df.loc[:, ~self_update_df.columns.isin(["savings_account_id_left"])]
    self_update_df.rename(columns={"savings_account_id_right":"savings_account_id"},inplace=True)
    data_collection["savings"]["data"] = pd.concat([create_df,self_update_df],axis=0).sort_values(
                                         by=["ts"],ascending=[True]).reset_index(
                                         drop=True)
    # data_collection["savings"]["data"].fillna(method="ffill", inplace=True)

    # Complete historical denormalized tables
    denom_account_card_df = join_table(data_collection["accounts"]["data"], 
                                       data_collection["cards"]["data"],
                                       "card_id", "card_id","left",
                                       suffix=("_account","_card"))
    denom_final_df = join_table(denom_account_card_df, data_collection["savings"]["data"],
                                "savings_account_id", "savings_account_id","left",
                                suffix=("","_saving"))
    
    # return denom_final_df.sort_values(by=["ts_account","ts","ts_card"],ascending=[True, True, True]).reset_index(drop=True)
    denom_final_df["ts_transaction"] = denom_final_df.apply (
                                       lambda row: row["ts"] if not pd.isnull(row["ts"]) else row["ts_card"], 
                                       axis=1)

    # denom_final_df.sort_values(by=["ts_account","ts_transaction"],ascending=[True, True]).reset_index(drop=True)

    denom_final_df["transaction_val"] = denom_final_df.apply(lambda x:add_value(x["credit_used"],x["balance"]),axis=1)

    valid_transaction_df = denom_final_df[[
        "id_account","id_card","id","op_card","op","credit_used","balance",
        "transaction_val","ts_transaction"]][
            ~pd.isnull(denom_final_df["ts_transaction"]) & ~pd.isnull(denom_final_df["transaction_val"])].sort_values(
                by=["ts_transaction"],ascending=[True]).reset_index(drop=True)
    
    valid_transaction_df = valid_transaction_df[(valid_transaction_df["op_card"] == "u") | (valid_transaction_df["op"] == "u")].reset_index(drop=True)

    vol_transaction_df = valid_transaction_df.groupby(
        by=["ts_transaction"], axis=0,dropna=True)["id_account"].size().reset_index(name='transaction_count')
    # transaction_stamp_df = denom_final_df.sort_values(by=["ts_transaction"],
    #                                ascending=[True]).reset_index(
    #                                drop=True)[["ts_account", "ts", "ts_card","account_id"]]
    
    # vol_transaction_df = transaction_stamp_df.groupby(by=["ts_transaction"], axis=0,dropna=True)[
    #                                             "account_id"].size().reset_index(
    #                                              name='transaction_count')
    
    # transaction_value_df = denom_final_df[["id_account","id_card","id", "creadit_used","balance","ts_transaction"]]

    # return denom_final_df
    # return data_collection
    # Fill Unknown
    data_collection["accounts"]["data"].fillna(method="ffill",inplace=True)
    data_collection["savings"]["data"].fillna(method="ffill", inplace=True)
    data_collection["cards"]["data"].fillna(method="ffill",inplace=True)
    # denom_final_df.fillna(method="ffill", inplace=True)
    # denom_final_df.fillna(method="ffill",inplace=True)
    # valid_transaction_df.fillna(method="ffill",inplace=True)
    # vol_transaction_df.fillna(method="ffill",inplace=True)

    # Transaction data
    print("Accounts Table")
    display(data_collection["accounts"]["data"])
    print("Savings Account Table")
    display(data_collection["savings"]["data"])
    print("Cards Table")
    display(data_collection["cards"]["data"])
    print("Complete Denormalized Tables")
    display(denom_final_df)
    print("Transaction Tables")
    display(valid_transaction_df)
    display(vol_transaction_df)

In [40]:
data = main()
# data.sort_values(by=["ts_account","ts_transaction"],ascending=[True, True]).reset_index(drop=True)

Start working  data/cards/
Done working  data/cards/
Start working  data/accounts/
Done working  data/accounts/
Start working  data/savings_accounts/
Done working  data/savings_accounts/
Accounts Table


,id,op,ts,data,account_id,name,address,phone_number,email,card_id,savings_account_id
0,a1globalid,c,2020-01-01 14:30:00,"{'account_id': 'a1', 'name': 'Anthony', 'addre...",a1,Anthony,New York,12345678,anthony@somebank.com,NaN,NaN
1,a1globalid,u,2020-01-01 15:00:00,{'phone_number': '87654321'},a1,Anthony,New York,87654321,anthony@somebank.com,NaN,NaN
2,a1globalid,u,2020-01-01 22:00:00,{'savings_account_id': 'sa1'},a1,Anthony,New York,87654321,anthony@somebank.com,NaN,sa1
3,a1globalid,u,2020-01-01 23:00:00,"{'address': 'Jakarta', 'email': 'anthony@anoth...",a1,Anthony,Jakarta,87654321,anthony@anotherbank.com,NaN,sa1
4,a1globalid,u,2020-01-02 08:00:00,{'card_id': 'c1'},a1,Anthony,Jakarta,87654321,anthony@anotherbank.com,c1,sa1
5,a1globalid,u,2020-01-15 16:01:00,{'card_id': ''},a1,Anthony,Jakarta,87654321,anthony@anotherbank.com,,sa1
6,a1globalid,u,2020-01-16 15:30:00,{'card_id': 'c2'},a1,Anthony,Jakarta,87654321,anthony@anotherbank.com,c2,sa1


Savings Account Table


,id,op,ts,data,savings_account_id,balance,interest_rate_percent,status
0,sa1globalid,c,2020-01-01 22:00:00,"{'savings_account_id': 'sa1', 'balance': 0, 'i...",sa1,0.0,1.5,ACTIVE
1,sa1globalid,u,2020-01-02 16:00:00,{'balance': 15000},sa1,15000.0,1.5,ACTIVE
2,sa1globalid,u,2020-01-05 00:31:00,{'interest_rate_percent': 3.0},sa1,15000.0,3.0,ACTIVE
3,sa1globalid,u,2020-01-10 16:30:00,{'balance': 40000},sa1,40000.0,3.0,ACTIVE
4,sa1globalid,u,2020-01-10 18:00:00,{'balance': 21000},sa1,21000.0,3.0,ACTIVE
5,sa1globalid,u,2020-01-15 16:01:00,{'interest_rate_percent': 1.5},sa1,21000.0,1.5,ACTIVE
6,sa1globalid,u,2020-01-18 05:01:00,{'interest_rate_percent': 4.0},sa1,21000.0,4.0,ACTIVE
7,sa1globalid,u,2020-01-20 14:30:00,{'balance': 33000},sa1,33000.0,4.0,ACTIVE


Cards Table


...

Complete Denormalized Tables


,id_account,op_account,ts_account,data_account,account_id,name,address,phone_number,email,card_id,...,status,id,op,ts,data,balance,interest_rate_percent,status_saving,ts_transaction,transaction_val
0,a1globalid,c,2020-01-01 14:30:00,"{'account_id': 'a1', 'name': 'Anthony', 'addre...",a1,Anthony,New York,12345678,anthony@somebank.com,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaN
1,a1globalid,u,2020-01-01 15:00:00,{'phone_number': '87654321'},a1,NaN,NaN,87654321,NaN,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaN
2,a1globalid,u,2020-01-01 22:00:00,{'savings_account_id': 'sa1'},a1,NaN,NaN,NaN,NaN,NaN,...,NaN,sa1globalid,c,2020-01-01 22:00:00,"{'savings_account_id': 'sa1', 'balance': 0, 'i...",0.0,1.5,ACTIVE,2020-01-01 22:00:00,0.0
3,a1globalid,u,2020-01-01 22:00:00,{'savings_account_id': 'sa1'},a1,NaN,NaN,NaN,NaN,NaN,...,NaN,sa1globalid,u,2020-01-02 16:00:00,{'balance': 15000},15000.0,NaN,NaN,2020-01-02 16:00:00,15000.0
4,a1globalid,u,2020-01-01 22:00:00,{'savings_account_id': 'sa1'},a1,NaN,NaN,NaN,NaN,NaN,...,NaN,sa1globalid,u,2020-01-05 00:31:00,{'interest_rate_percent': 3.0},NaN,3.0,NaN,2020-01-05 00:31:00,NaN
5,a1globalid,u,2020-01-01 22:00:00,{'savings_account_id': 'sa1'},a1,NaN,NaN,NaN,NaN,NaN,...,NaN,sa1globalid,u,2020-01-10 16:30:00,{'balance': 40000},40000.0,NaN,NaN,2020-01-10 16:30:00,40000.0
6,a1globalid,u,2020-01-01 22:00:00,{'savings_account_id': 'sa1'},a1,NaN,NaN,NaN,NaN,NaN,...,NaN,sa1globalid,u,2020-01-10 18:00:00,{'balance': 21000},21000.0,NaN,NaN,2020-01-10 18:00:00,21000.0
7,a1globalid,u,2020-01-01 22:00:00,{'savings_account_id': 'sa1'},a1,NaN,NaN,NaN,NaN,NaN,...,NaN,sa1globalid,u,2020-01-15 16:01:00,{'interest_rate_percent': 1.5},NaN,1.5,NaN,2020-01-15 16:01:00,NaN
8,a1globalid,u,2020-01-01 22:00:00,{'savings_account_id': 'sa1'},a1,NaN,NaN,NaN,NaN,NaN,...,NaN,sa1globalid,u,2020-01-18 05:01:00,{'interest_rate_percent': 4.0},NaN,4.0,NaN,2020-01-18 05:01:00,NaN
9,a1globalid,u,2020-01-01 22:00:00,{'savings_account_id': 'sa1'},a1,NaN,NaN,NaN,NaN,NaN,...,NaN,sa1globalid,u,2020-01-20 14:30:00,{'balance': 33000},33000.0,NaN,NaN,2020-01-20 14:30:00,33000.0


Transaction Tables


,id_account,id_card,id,op_card,op,credit_used,balance,transaction_val,ts_transaction
0,a1globalid,NaN,sa1globalid,NaN,u,NaN,15000.0,15000.0,2020-01-02 16:00:00
1,a1globalid,c1globalid,NaN,u,NaN,12000.0,NaN,12000.0,2020-01-06 19:30:00
2,a1globalid,c1globalid,NaN,u,NaN,19000.0,NaN,19000.0,2020-01-08 01:00:00
3,a1globalid,NaN,sa1globalid,NaN,u,NaN,40000.0,40000.0,2020-01-10 16:30:00
4,a1globalid,NaN,sa1globalid,NaN,u,NaN,21000.0,21000.0,2020-01-10 18:00:00
5,a1globalid,c1globalid,NaN,u,NaN,0.0,NaN,0.0,2020-01-10 18:00:00
6,a1globalid,c2globalid,NaN,u,NaN,37000.0,NaN,37000.0,2020-01-18 22:30:00
7,a1globalid,NaN,sa1globalid,NaN,u,NaN,33000.0,33000.0,2020-01-20 14:30:00


,ts_transaction,transaction_count
0,2020-01-02 16:00:00,1
1,2020-01-06 19:30:00,1
2,2020-01-08 01:00:00,1
3,2020-01-10 16:30:00,1
4,2020-01-10 18:00:00,2
5,2020-01-18 22:30:00,1
6,2020-01-20 14:30:00,1


In [129]:
data.columns

Index(['id_account', 'op_account', 'ts_account', 'data_account', 'account_id',
       'name', 'address', 'phone_number', 'email', 'card_id',
       'savings_account_id', 'id_card', 'op_card', 'ts_card', 'data_card',
       'card_number', 'credit_used', 'monthly_limit', 'status', 'id', 'op',
       'ts', 'data', 'balance', 'interest_rate_percent', 'status_saving',
       'ts_transaction'],
      dtype='object')

In [16]:
valid_transaction = data[["id_account","id_card","id", "credit_used","balance","transaction_val","ts_transaction"]][~pd.isnull(data["ts_transaction"]) & ~pd.isnull(data["transaction_val"])].sort_values(by=["ts_transaction"],ascending=[True]).reset_index(drop=True)

In [17]:
valid_transaction

,id_account,id_card,id,credit_used,balance,transaction_val,ts_transaction
0,a1globalid,NaN,sa1globalid,NaN,0.0,0.0,2020-01-01 22:00:00
1,a1globalid,c1globalid,NaN,0.0,NaN,0.0,2020-01-02 08:00:00
2,a1globalid,NaN,sa1globalid,NaN,15000.0,15000.0,2020-01-02 16:00:00
3,a1globalid,c1globalid,NaN,12000.0,NaN,12000.0,2020-01-06 19:30:00
4,a1globalid,c1globalid,NaN,19000.0,NaN,19000.0,2020-01-08 01:00:00
5,a1globalid,NaN,sa1globalid,NaN,40000.0,40000.0,2020-01-10 16:30:00
6,a1globalid,NaN,sa1globalid,NaN,21000.0,21000.0,2020-01-10 18:00:00
7,a1globalid,c1globalid,NaN,0.0,NaN,0.0,2020-01-10 18:00:00
8,a1globalid,c2globalid,NaN,0.0,NaN,0.0,2020-01-16 15:30:00
9,a1globalid,c2globalid,NaN,37000.0,NaN,37000.0,2020-01-18 22:30:00


In [13]:
vol_transact = valid_transaction.groupby(by=["ts_transaction"], axis=0,dropna=True)["id_account"].size().reset_index(name='transaction_count')

In [14]:
vol_transact

,ts_transaction,transaction_count
0,2020-01-01 22:00:00,1
1,2020-01-02 08:00:00,1
2,2020-01-02 16:00:00,1
3,2020-01-06 19:30:00,1
4,2020-01-08 01:00:00,1
5,2020-01-10 16:30:00,1
6,2020-01-10 18:00:00,2
7,2020-01-16 15:30:00,1
8,2020-01-18 22:30:00,1
9,2020-01-20 14:30:00,1


In [10]:
def add_value(x:int,y:int)->int:
    if pd.isna(x) and pd.isna(y):
        return np.nan
    else:
        return (x if not pd.isna(x) else 0) + (y if not pd.isna(y) else 0)

# data["transaction_val"] = data.apply(lambda x:(x["credit_used"] if not pd.isna(x["credit_used"]) else 0) + 
#                                     (x["balance"] if not pd.isna(x["balance"]) else 0),axis=1)
data["transaction_val"] = data.apply(lambda x:add_value(x["credit_used"],x["balance"]),axis=1) 

In [62]:
data.to_excel("denom_final.xlsx")

In [117]:
transaction = data.sort_values(by=["ts_transaction"],ascending=[True]).reset_index(drop=True)[["ts_account", "ts", "ts_card","ts_transaction","data_account","data_card","data","account_id"]]

In [118]:
transaction

,ts_account,ts,ts_card,ts_transaction,data_account,data_card,data,account_id
0,2020-01-01 22:00:00,2020-01-01 22:00:00,NaT,2020-01-01 22:00:00,{'savings_account_id': 'sa1'},NaN,"{'savings_account_id': 'sa1', 'balance': 0, 'i...",a1
1,2020-01-02 08:00:00,NaT,2020-01-02 08:00:00,2020-01-02 08:00:00,{'card_id': 'c1'},"{'card_id': 'c1', 'card_number': '11112222', '...",NaN,a1
2,2020-01-01 22:00:00,2020-01-02 16:00:00,NaT,2020-01-02 16:00:00,{'savings_account_id': 'sa1'},NaN,{'balance': 15000},a1
3,2020-01-02 08:00:00,NaT,2020-01-05 00:30:00,2020-01-05 00:30:00,{'card_id': 'c1'},{'status': 'ACTIVE'},NaN,a1
4,2020-01-01 22:00:00,2020-01-05 00:31:00,NaT,2020-01-05 00:31:00,{'savings_account_id': 'sa1'},NaN,{'interest_rate_percent': 3.0},a1
5,2020-01-02 08:00:00,NaT,2020-01-06 19:30:00,2020-01-06 19:30:00,{'card_id': 'c1'},{'credit_used': 12000},NaN,a1
6,2020-01-02 08:00:00,NaT,2020-01-08 01:00:00,2020-01-08 01:00:00,{'card_id': 'c1'},{'credit_used': 19000},NaN,a1
7,2020-01-01 22:00:00,2020-01-10 16:30:00,NaT,2020-01-10 16:30:00,{'savings_account_id': 'sa1'},NaN,{'balance': 40000},a1
8,2020-01-01 22:00:00,2020-01-10 18:00:00,NaT,2020-01-10 18:00:00,{'savings_account_id': 'sa1'},NaN,{'balance': 21000},a1
9,2020-01-02 08:00:00,NaT,2020-01-10 18:00:00,2020-01-10 18:00:00,{'card_id': 'c1'},{'credit_used': 0},NaN,a1


In [125]:
vol_transact = transaction.groupby(by=["ts_transaction"], axis=0,dropna=True)["account_id"].size().reset_index(name='transaction_count')

In [128]:
sum(vol_transact["transaction_count"].to_list())

17

In [107]:
data[["ts_account", "ts", "ts_card","ts_transaction","data_account","data_card","data"]]

,ts_account,ts,ts_card,ts_transaction,data_account,data_card,data
0,2020-01-01 14:30:00,NaT,NaT,NaT,"{'account_id': 'a1', 'name': 'Anthony', 'addre...",NaN,NaN
1,2020-01-01 15:00:00,NaT,NaT,NaT,{'phone_number': '87654321'},NaN,NaN
2,2020-01-01 22:00:00,2020-01-01 22:00:00,NaT,2020-01-01 22:00:00,{'savings_account_id': 'sa1'},NaN,"{'savings_account_id': 'sa1', 'balance': 0, 'i..."
3,2020-01-01 22:00:00,2020-01-02 16:00:00,NaT,2020-01-02 16:00:00,{'savings_account_id': 'sa1'},NaN,{'balance': 15000}
4,2020-01-01 22:00:00,2020-01-05 00:31:00,NaT,2020-01-05 00:31:00,{'savings_account_id': 'sa1'},NaN,{'interest_rate_percent': 3.0}
5,2020-01-01 22:00:00,2020-01-10 16:30:00,NaT,2020-01-10 16:30:00,{'savings_account_id': 'sa1'},NaN,{'balance': 40000}
6,2020-01-01 22:00:00,2020-01-10 18:00:00,NaT,2020-01-10 18:00:00,{'savings_account_id': 'sa1'},NaN,{'balance': 21000}
7,2020-01-01 22:00:00,2020-01-15 16:01:00,NaT,2020-01-15 16:01:00,{'savings_account_id': 'sa1'},NaN,{'interest_rate_percent': 1.5}
8,2020-01-01 22:00:00,2020-01-18 05:01:00,NaT,2020-01-18 05:01:00,{'savings_account_id': 'sa1'},NaN,{'interest_rate_percent': 4.0}
9,2020-01-01 22:00:00,2020-01-20 14:30:00,NaT,2020-01-20 14:30:00,{'savings_account_id': 'sa1'},NaN,{'balance': 33000}


In [95]:
data["transaction_ts"] = data.apply (lambda row: row["ts"] if not pd.isnull(row["ts"]) else row["ts_card"], axis=1)

In [83]:
denom_account_card_df = join_table(data["accounts"]["data"], 
                                       data["cards"]["data"],
                                       "card_id", "card_id","left",
                                       suffix=("_account","_card"))

In [84]:
denom_account_card_df

,id_account,op_account,ts_account,data_account,account_id,name,address,phone_number,email,card_id,savings_account_id,id_card,op_card,ts_card,data_card,card_number,credit_used,monthly_limit,status
0,a1globalid,c,2020-01-01 14:30:00,"{'account_id': 'a1', 'name': 'Anthony', 'addre...",a1,Anthony,New York,12345678,anthony@somebank.com,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
1,a1globalid,u,2020-01-01 15:00:00,{'phone_number': '87654321'},a1,NaN,NaN,87654321,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
2,a1globalid,u,2020-01-01 22:00:00,{'savings_account_id': 'sa1'},a1,NaN,NaN,NaN,NaN,NaN,sa1,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
3,a1globalid,u,2020-01-01 23:00:00,"{'address': 'Jakarta', 'email': 'anthony@anoth...",a1,NaN,Jakarta,NaN,anthony@anotherbank.com,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
4,a1globalid,u,2020-01-02 08:00:00,{'card_id': 'c1'},a1,NaN,NaN,NaN,NaN,c1,NaN,c1globalid,c,2020-01-02 08:00:00,"{'card_id': 'c1', 'card_number': '11112222', '...",11112222,0.0,30000.0,PENDING
5,a1globalid,u,2020-01-02 08:00:00,{'card_id': 'c1'},a1,NaN,NaN,NaN,NaN,c1,NaN,c1globalid,u,2020-01-05 00:30:00,{'status': 'ACTIVE'},11112222,NaN,NaN,ACTIVE
6,a1globalid,u,2020-01-02 08:00:00,{'card_id': 'c1'},a1,NaN,NaN,NaN,NaN,c1,NaN,c1globalid,u,2020-01-06 19:30:00,{'credit_used': 12000},11112222,12000.0,NaN,NaN
7,a1globalid,u,2020-01-02 08:00:00,{'card_id': 'c1'},a1,NaN,NaN,NaN,NaN,c1,NaN,c1globalid,u,2020-01-08 01:00:00,{'credit_used': 19000},11112222,19000.0,NaN,NaN
8,a1globalid,u,2020-01-02 08:00:00,{'card_id': 'c1'},a1,NaN,NaN,NaN,NaN,c1,NaN,c1globalid,u,2020-01-10 18:00:00,{'credit_used': 0},11112222,0.0,NaN,NaN
9,a1globalid,u,2020-01-02 08:00:00,{'card_id': 'c1'},a1,NaN,NaN,NaN,NaN,c1,NaN,c1globalid,u,2020-01-15 16:00:00,{'status': 'CLOSED'},11112222,NaN,NaN,CLOSED


In [87]:
denom_account_saving_df = join_table(denom_account_card_df, 
                                       data["savings"]["data"],
                                       "savings_account_id", "savings_account_id","left",
                                       suffix=("","_saving"))

In [90]:
denom_account_saving_df.columns

Index(['id_account', 'op_account', 'ts_account', 'data_account', 'account_id',
       'name', 'address', 'phone_number', 'email', 'card_id',
       'savings_account_id', 'id_card', 'op_card', 'ts_card', 'data_card',
       'card_number', 'credit_used', 'monthly_limit', 'status', 'id', 'op',
       'ts', 'data', 'balance', 'interest_rate_percent', 'status_saving'],
      dtype='object')

In [86]:
# denom_account_saving_df
data["savings"]["data"]

,id,op,ts,data,savings_account_id,balance,interest_rate_percent,status
0,sa1globalid,c,2020-01-01 22:00:00,"{'savings_account_id': 'sa1', 'balance': 0, 'i...",sa1,0.0,1.5,ACTIVE
1,sa1globalid,u,2020-01-02 16:00:00,{'balance': 15000},sa1,15000.0,NaN,NaN
2,sa1globalid,u,2020-01-05 00:31:00,{'interest_rate_percent': 3.0},sa1,NaN,3.0,NaN
3,sa1globalid,u,2020-01-10 16:30:00,{'balance': 40000},sa1,40000.0,NaN,NaN
4,sa1globalid,u,2020-01-10 18:00:00,{'balance': 21000},sa1,21000.0,NaN,NaN
5,sa1globalid,u,2020-01-15 16:01:00,{'interest_rate_percent': 1.5},sa1,NaN,1.5,NaN
6,sa1globalid,u,2020-01-18 05:01:00,{'interest_rate_percent': 4.0},sa1,NaN,4.0,NaN
7,sa1globalid,u,2020-01-20 14:30:00,{'balance': 33000},sa1,33000.0,NaN,NaN


In [65]:
for key, val in data_collection.items():
    data_collection[key]["data"] = load_data(val["url"],{"ts": ts_mikro})

Start working  data/cards/
Done working  data/cards/
Start working  data/accounts/
Done working  data/accounts/
Start working  data/savings_accounts/
Done working  data/savings_accounts/


In [5]:
data_collection["savings"]["data"]

,id,op,ts,data,savings_account_id,balance,interest_rate_percent,status
0,sa1globalid,c,2020-01-01 22:00:00,"{'savings_account_id': 'sa1', 'balance': 0, 'i...",sa1,0.0,1.5,ACTIVE
1,sa1globalid,u,2020-01-20 14:30:00,{'balance': 33000},NaN,33000.0,NaN,NaN
2,sa1globalid,u,2020-01-02 16:00:00,{'balance': 15000},NaN,15000.0,NaN,NaN
3,sa1globalid,u,2020-01-15 16:01:00,{'interest_rate_percent': 1.5},NaN,NaN,1.5,NaN
4,sa1globalid,u,2020-01-10 16:30:00,{'balance': 40000},NaN,40000.0,NaN,NaN
5,sa1globalid,u,2020-01-05 00:31:00,{'interest_rate_percent': 3.0},NaN,NaN,3.0,NaN
6,sa1globalid,u,2020-01-10 18:00:00,{'balance': 21000},NaN,21000.0,NaN,NaN
7,sa1globalid,u,2020-01-18 05:01:00,{'interest_rate_percent': 4.0},NaN,NaN,4.0,NaN


In [45]:
data_collection["accounts"]["data"].sort_values(by=["ts"],ascending=[True])

,id,op,ts,data,account_id,name,address,phone_number,email,card_id,savings_account_id
0,a1globalid,c,2020-01-01 14:30:00,"{'account_id': 'a1', 'name': 'Anthony', 'addre...",a1,Anthony,New York,12345678,anthony@somebank.com,NaN,NaN
5,a1globalid,u,2020-01-01 15:00:00,{'phone_number': '87654321'},NaN,NaN,NaN,87654321,NaN,NaN,NaN
6,a1globalid,u,2020-01-01 22:00:00,{'savings_account_id': 'sa1'},NaN,NaN,NaN,NaN,NaN,NaN,sa1
1,a1globalid,u,2020-01-01 23:00:00,"{'address': 'Jakarta', 'email': 'anthony@anoth...",NaN,NaN,Jakarta,NaN,anthony@anotherbank.com,NaN,NaN
3,a1globalid,u,2020-01-02 08:00:00,{'card_id': 'c1'},NaN,NaN,NaN,NaN,NaN,c1,NaN
2,a1globalid,u,2020-01-15 16:01:00,{'card_id': ''},NaN,NaN,NaN,NaN,NaN,,NaN
4,a1globalid,u,2020-01-16 15:30:00,{'card_id': 'c2'},NaN,NaN,NaN,NaN,NaN,c2,NaN


In [7]:
data_collection["cards"]["data"]

,id,op,ts,data,card_id,card_number,credit_used,monthly_limit,status
0,c1globalid,c,2020-01-02 08:00:00,"{'card_id': 'c1', 'card_number': '11112222', '...",c1,11112222,0.0,30000.0,PENDING
1,c2globalid,c,2020-01-16 15:30:00,"{'card_id': 'c2', 'card_number': '12123434', '...",c2,12123434,0.0,70000.0,PENDING
2,c2globalid,u,2020-01-18 05:00:00,{'status': 'ACTIVE'},NaN,NaN,NaN,NaN,ACTIVE
3,c1globalid,u,2020-01-08 01:00:00,{'credit_used': 19000},NaN,NaN,19000.0,NaN,NaN
4,c1globalid,u,2020-01-05 00:30:00,{'status': 'ACTIVE'},NaN,NaN,NaN,NaN,ACTIVE
5,c1globalid,u,2020-01-06 19:30:00,{'credit_used': 12000},NaN,NaN,12000.0,NaN,NaN
6,c2globalid,u,2020-01-18 22:30:00,{'credit_used': 37000},NaN,NaN,37000.0,NaN,NaN
7,c1globalid,u,2020-01-15 16:00:00,{'status': 'CLOSED'},NaN,NaN,NaN,NaN,CLOSED
8,c1globalid,u,2020-01-10 18:00:00,{'credit_used': 0},NaN,NaN,0.0,NaN,NaN


In [17]:
create_df = data_collection["savings"]["data"][data_collection["savings"]["data"]["op"] == "c"]
update_df = data_collection["savings"]["data"][data_collection["savings"]["data"]["op"] == "u"]
self_update_df = join_table(update_df,create_df[["id","savings_account_id"]],"id","id",how="left")
self_update_df = self_update_df.loc[:, ~self_update_df.columns.isin(["savings_account_id_left"])]
self_update_df.rename(columns={"savings_account_id_right":"savings_account_id"},inplace=True)
data_collection["accounts"]["data"] = pd.concat([create_df,self_update_df],axis=0).reset_index(drop=True)
# self_update_df

,id,op,ts,data,balance,interest_rate_percent,status,savings_account_id
0,sa1globalid,u,2020-01-20 14:30:00,{'balance': 33000},33000.0,NaN,NaN,sa1
1,sa1globalid,u,2020-01-02 16:00:00,{'balance': 15000},15000.0,NaN,NaN,sa1
2,sa1globalid,u,2020-01-15 16:01:00,{'interest_rate_percent': 1.5},NaN,1.5,NaN,sa1
3,sa1globalid,u,2020-01-10 16:30:00,{'balance': 40000},40000.0,NaN,NaN,sa1
4,sa1globalid,u,2020-01-05 00:31:00,{'interest_rate_percent': 3.0},NaN,3.0,NaN,sa1
5,sa1globalid,u,2020-01-10 18:00:00,{'balance': 21000},21000.0,NaN,NaN,sa1
6,sa1globalid,u,2020-01-18 05:01:00,{'interest_rate_percent': 4.0},NaN,4.0,NaN,sa1


In [16]:
card = data_collection["accounts"]["data"]
create_card_df = card[card["op"] == "c"]
update_card_df = card[card["op"] == "u"]
self_card_df = join_table(update_card_df,create_card_df[["id","account_id"]],"id","id",how="left")
self_card_df = self_card_df.loc[:, ~self_card_df.columns.isin(["account_id_left"])]
self_card_df.rename(columns={"account_id_right":"account_id"},inplace=True)
final_df = pd.concat([create_card_df,self_card_df],axis=0).reset_index(drop=True)
final_df

,id,op,ts,data,account_id,name,address,phone_number,email,card_id,savings_account_id
0,a1globalid,c,2020-01-01 14:30:00,"{'account_id': 'a1', 'name': 'Anthony', 'addre...",a1,Anthony,New York,12345678,anthony@somebank.com,NaN,NaN
1,a1globalid,u,2020-01-01 23:00:00,"{'address': 'Jakarta', 'email': 'anthony@anoth...",a1,NaN,Jakarta,NaN,anthony@anotherbank.com,NaN,NaN
2,a1globalid,u,2020-01-15 16:01:00,{'card_id': ''},a1,NaN,NaN,NaN,NaN,,NaN
3,a1globalid,u,2020-01-02 08:00:00,{'card_id': 'c1'},a1,NaN,NaN,NaN,NaN,c1,NaN
4,a1globalid,u,2020-01-16 15:30:00,{'card_id': 'c2'},a1,NaN,NaN,NaN,NaN,c2,NaN
5,a1globalid,u,2020-01-01 15:00:00,{'phone_number': '87654321'},a1,NaN,NaN,87654321,NaN,NaN,NaN
6,a1globalid,u,2020-01-01 22:00:00,{'savings_account_id': 'sa1'},a1,NaN,NaN,NaN,NaN,NaN,sa1


In [14]:
card = data_collection["cards"]["data"]
create_card_df = card[card["op"] == "c"]
update_card_df = card[card["op"] == "u"]
self_card_df = join_table(update_card_df,create_card_df[["id","card_id","card_number"]],"id","id",how="left")
self_card_df = self_card_df.loc[:, ~self_card_df.columns.isin(["card_id_left","card_number_left"])]
self_card_df.rename(columns={"card_id_right":"card_id","card_number_right":"card_number"},inplace=True)
final_df = pd.concat([create_card_df,self_card_df],axis=0).reset_index(drop=True)
final_df

,id,op,ts,data,card_id,card_number,credit_used,monthly_limit,status
0,c1globalid,c,2020-01-02 08:00:00,"{'card_id': 'c1', 'card_number': '11112222', '...",c1,11112222,0.0,30000.0,PENDING
1,c2globalid,c,2020-01-16 15:30:00,"{'card_id': 'c2', 'card_number': '12123434', '...",c2,12123434,0.0,70000.0,PENDING
2,c2globalid,u,2020-01-18 05:00:00,{'status': 'ACTIVE'},c2,12123434,NaN,NaN,ACTIVE
3,c1globalid,u,2020-01-08 01:00:00,{'credit_used': 19000},c1,11112222,19000.0,NaN,NaN
4,c1globalid,u,2020-01-05 00:30:00,{'status': 'ACTIVE'},c1,11112222,NaN,NaN,ACTIVE
5,c1globalid,u,2020-01-06 19:30:00,{'credit_used': 12000},c1,11112222,12000.0,NaN,NaN
6,c2globalid,u,2020-01-18 22:30:00,{'credit_used': 37000},c2,12123434,37000.0,NaN,NaN
7,c1globalid,u,2020-01-15 16:00:00,{'status': 'CLOSED'},c1,11112222,NaN,NaN,CLOSED
8,c1globalid,u,2020-01-10 18:00:00,{'credit_used': 0},c1,11112222,0.0,NaN,NaN


In [15]:
final_df["data"].to_list()

[{'card_id': 'c1',
  'card_number': '11112222',
  'credit_used': 0,
  'monthly_limit': 30000,
  'status': 'PENDING'},
 {'card_id': 'c2',
  'card_number': '12123434',
  'credit_used': 0,
  'monthly_limit': 70000,
  'status': 'PENDING'},
 {'status': 'ACTIVE'},
 {'credit_used': 19000},
 {'status': 'ACTIVE'},
 {'credit_used': 12000},
 {'credit_used': 37000},
 {'status': 'CLOSED'},
 {'credit_used': 0}]

In [6]:
def join_table(left_df:pd.DataFrame, right_df:pd.DataFrame, 
               left_on:str,right_on:str, how:str="inner",
               suffix:tuple=('_left', '_right')) -> pd.DataFrame:
    return pd.merge(left_df,right_df,left_on=left_on,right_on=right_on,how=how,suffixes=suffix)

In [33]:
accounts = data["accounts"]["data"]
savings = data["savings"]["data"]
cards = data["cards"]["data"]

account_card_df = join_table(accounts, cards,"card_id", "card_id","left",suffix=("_account","_card"))
join_saving_df = join_table(account_card_df, savings,"savings_account_id", "savings_account_id","left",suffix=("","_saving")) 

In [36]:
join_saving_df

,id_account,op_account,ts_account,data_account,account_id,name,address,phone_number,email,card_id,...,credit_used,monthly_limit,status,id,op,ts,data,balance,interest_rate_percent,status_saving
0,a1globalid,c,2020-01-01 14:30:00,"{'account_id': 'a1', 'name': 'Anthony', 'addre...",a1,Anthony,New York,12345678,anthony@somebank.com,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1,a1globalid,u,2020-01-01 23:00:00,"{'address': 'Jakarta', 'email': 'anthony@anoth...",a1,NaN,Jakarta,NaN,anthony@anotherbank.com,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2,a1globalid,u,2020-01-15 16:01:00,{'card_id': ''},a1,NaN,NaN,NaN,NaN,,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
3,a1globalid,u,2020-01-02 08:00:00,{'card_id': 'c1'},a1,NaN,NaN,NaN,NaN,c1,...,0.0,30000.0,PENDING,NaN,NaN,NaT,NaN,NaN,NaN,NaN
4,a1globalid,u,2020-01-02 08:00:00,{'card_id': 'c1'},a1,NaN,NaN,NaN,NaN,c1,...,19000.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
5,a1globalid,u,2020-01-02 08:00:00,{'card_id': 'c1'},a1,NaN,NaN,NaN,NaN,c1,...,NaN,NaN,ACTIVE,NaN,NaN,NaT,NaN,NaN,NaN,NaN
6,a1globalid,u,2020-01-02 08:00:00,{'card_id': 'c1'},a1,NaN,NaN,NaN,NaN,c1,...,12000.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
7,a1globalid,u,2020-01-02 08:00:00,{'card_id': 'c1'},a1,NaN,NaN,NaN,NaN,c1,...,NaN,NaN,CLOSED,NaN,NaN,NaT,NaN,NaN,NaN,NaN
8,a1globalid,u,2020-01-02 08:00:00,{'card_id': 'c1'},a1,NaN,NaN,NaN,NaN,c1,...,0.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
9,a1globalid,u,2020-01-16 15:30:00,{'card_id': 'c2'},a1,NaN,NaN,NaN,NaN,c2,...,0.0,70000.0,PENDING,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [47]:
# savings.columns
a = {old:f"sav_{old}" for old in savings.columns}
print(a)

{'id': 'sav_id', 'op': 'sav_op', 'ts': 'sav_ts', 'data': 'sav_data', 'savings_account_id': 'sav_savings_account_id', 'balance': 'sav_balance', 'interest_rate_percent': 'sav_interest_rate_percent', 'status': 'sav_status'}


In [18]:
show_col = ["account_id","card_id", "data_account","data_card"]
account_card_df[show_col]

,account_id,card_id,data_account,data_card
0,a1,NaN,"{'account_id': 'a1', 'name': 'Anthony', 'addre...",{'status': 'ACTIVE'}
1,a1,NaN,"{'account_id': 'a1', 'name': 'Anthony', 'addre...",{'credit_used': 19000}
2,a1,NaN,"{'account_id': 'a1', 'name': 'Anthony', 'addre...",{'status': 'ACTIVE'}
3,a1,NaN,"{'account_id': 'a1', 'name': 'Anthony', 'addre...",{'credit_used': 12000}
4,a1,NaN,"{'account_id': 'a1', 'name': 'Anthony', 'addre...",{'credit_used': 37000}
5,a1,NaN,"{'account_id': 'a1', 'name': 'Anthony', 'addre...",{'status': 'CLOSED'}
6,a1,NaN,"{'account_id': 'a1', 'name': 'Anthony', 'addre...",{'credit_used': 0}
7,NaN,NaN,"{'address': 'Jakarta', 'email': 'anthony@anoth...",{'status': 'ACTIVE'}
8,NaN,NaN,"{'address': 'Jakarta', 'email': 'anthony@anoth...",{'credit_used': 19000}
9,NaN,NaN,"{'address': 'Jakarta', 'email': 'anthony@anoth...",{'status': 'ACTIVE'}


In [37]:
join_saving_df.head()

,id_account,op_account,ts_account,data_account,account_id,name,address,phone_number,email,card_id,...,credit_used,monthly_limit,status,id,op,ts,data,balance,interest_rate_percent,status_saving
0,a1globalid,c,2020-01-01 14:30:00,"{'account_id': 'a1', 'name': 'Anthony', 'addre...",a1,Anthony,New York,12345678,anthony@somebank.com,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1,a1globalid,u,2020-01-01 23:00:00,"{'address': 'Jakarta', 'email': 'anthony@anoth...",a1,Anthony,Jakarta,12345678,anthony@anotherbank.com,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2,a1globalid,u,2020-01-15 16:01:00,{'card_id': ''},a1,Anthony,Jakarta,12345678,anthony@anotherbank.com,,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
3,a1globalid,u,2020-01-02 08:00:00,{'card_id': 'c1'},a1,Anthony,Jakarta,12345678,anthony@anotherbank.com,c1,...,0.0,30000.0,PENDING,NaN,NaN,NaT,NaN,NaN,NaN,NaN
4,a1globalid,u,2020-01-16 15:30:00,{'card_id': 'c2'},a1,Anthony,Jakarta,12345678,anthony@anotherbank.com,c2,...,0.0,70000.0,PENDING,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [39]:
join_saving_df.to_excel("final.xlsx", index=False)

In [43]:
join_saving_df.columns,len(join_saving_df.columns)

(Index(['id_account', 'op_account', 'ts_account', 'data_account', 'account_id',
        'name', 'address', 'phone_number', 'email', 'card_id',
        'savings_account_id', 'id_card', 'op_card', 'ts_card', 'data_card',
        'card_number', 'credit_used', 'monthly_limit', 'status', 'id', 'op',
        'ts', 'data', 'balance', 'interest_rate_percent', 'status_saving'],
       dtype='object'),
 26)

In [ ]:
show_col = ["id_account", "account_id","id_card", "savings_account_id","op_account","op_card","ts_account","ts_card","name","email"]